### Instructions 

Before running this notebook, create a local environment that matches the environemnt on the cloud. Dask Cloud Provider also let's you specify a custom docker image to run on your cluster ([see Dask Cloud Provider's documentation](https://cloudprovider.dask.org/en/latest/aws.html#elastic-compute-cloud-ec2) for more details). This benchmark was run using [Dask 2024.9.1](https://hub.docker.com/r/daskdev/dask/tags).

You can use the provided `env.yml` to create your environment locally.

``` shell
cd benchmarks/dask
conda env create -f env.yml
```
This will create the conda environment `benchmark_dask`, attach it to this notebook.

In [1]:
import time

import dask.dataframe as dd
from dask.distributed import Client
from dask_cloudprovider.aws import EC2Cluster

In [2]:
env_vars = {"EXTRA_CONDA_PACKAGES": "s3fs==2024.10.0"}
cluster = EC2Cluster(
    # NOTE: Setting security = False to avoid large config size
    # https://github.com/dask/dask-cloudprovider/issues/249
    security=False,
    n_workers=4,
    scheduler_instance_type="c6i.xlarge",
    worker_instance_type="r6i.16xlarge",
    docker_image="daskdev/dask:2024.9.1-py3.10",
    # Profile with AmazonS3FullAccess
    iam_instance_profile={"Name": "dask-benchmark"},
    # Region for accessing bodo-example-data
    region="us-east-2",
    env_vars=env_vars,
    debug=True,
)

In [ ]:
client = Client(cluster)
print(client.dashboard_link)

In [33]:
# first parquet file
dataset = [
    f"s3://bodo-example-data/nyc-taxi/fhvhv_tripdata/fhvhv_tripdata_2019-{i:02}.parquet"
    for i in range(2, 8)
]

In [34]:
# run on entire dataset
dataset = "s3://bodo-example-data/nyc-taxi/fhvhv_tripdata/"

In [17]:
def read_pq():
    fhvhv_tripdata = dd.read_parquet(dataset)

    fhvhv_tripdata.to_parquet("s3://test-dask/fhvhv_tripdata.pq", compute=True)

    return fhvhv_tripdata.head(5)


future = client.submit(read_pq)
print(future.result())

  hvfhs_license_num dispatching_base_num originating_base_num  \
0            HV0003               B02867               B02867   
1            HV0003               B02879               B02879   
2            HV0005               B02510                 <NA>   
3            HV0005               B02510                 <NA>   
4            HV0005               B02510                 <NA>   

     request_datetime   on_scene_datetime     pickup_datetime  \
0 2019-02-01 00:01:26 2019-02-01 00:02:55 2019-02-01 00:05:18   
1 2019-02-01 00:26:08 2019-02-01 00:41:29 2019-02-01 00:41:29   
2 2019-02-01 00:48:58                 NaT 2019-02-01 00:51:34   
3 2019-02-01 00:02:15                 NaT 2019-02-01 00:03:51   
4 2019-02-01 00:06:17                 NaT 2019-02-01 00:09:44   

     dropoff_datetime  PULocationID  DOLocationID  trip_miles  ...  sales_tax  \
0 2019-02-01 00:14:57           245           251        2.45  ...       0.83   
1 2019-02-01 00:49:39           216           197       

In [35]:
def get_monthly_travels_weather():
    start = time.time()
    central_park_weather_observations = dd.read_csv(
        "s3://bodo-example-data/nyc-taxi/central_park_weather.csv",
        parse_dates=["DATE"],
        storage_options={"anon": True},
    )
    central_park_weather_observations = central_park_weather_observations.rename(
        columns={"DATE": "date", "PRCP": "precipitation"}
    )

    fhvhv_tripdata = dd.read_parquet(dataset, storage_options={"anon": True})

    central_park_weather_observations["date"] = central_park_weather_observations[
        "date"
    ].dt.date
    fhvhv_tripdata["date"] = fhvhv_tripdata["pickup_datetime"].dt.date
    fhvhv_tripdata["month"] = fhvhv_tripdata["pickup_datetime"].dt.month
    fhvhv_tripdata["hour"] = fhvhv_tripdata["pickup_datetime"].dt.hour
    fhvhv_tripdata["weekday"] = fhvhv_tripdata["pickup_datetime"].dt.dayofweek.isin(
        [0, 1, 2, 3, 4]
    )
    monthly_trips_weather = fhvhv_tripdata.merge(
        central_park_weather_observations, on="date", how="inner"
    )
    monthly_trips_weather["date_with_precipitation"] = (
        monthly_trips_weather["precipitation"] > 0.1
    )

    def get_time_bucket(t):
        bucket = "other"
        if t in (8, 9, 10):
            bucket = "morning"
        elif t in (11, 12, 13, 14, 15):
            bucket = "midday"
        elif t in (16, 17, 18):
            bucket = "afternoon"
        elif t in (19, 20, 21):
            bucket = "evening"
        return bucket

    monthly_trips_weather["time_bucket"] = monthly_trips_weather.hour.map(
        get_time_bucket, meta=("hour", "object")
    )
    monthly_trips_weather = (
        monthly_trips_weather.groupby(
            [
                "PULocationID",
                "DOLocationID",
                "month",
                "weekday",
                "date_with_precipitation",
                "time_bucket",
            ],
        )
        .agg({"hvfhs_license_num": "count", "trip_miles": "mean"})
        .reset_index()
    )
    monthly_trips_weather = monthly_trips_weather.sort_values(
        by=[
            "PULocationID",
            "DOLocationID",
            "month",
            "weekday",
            "date_with_precipitation",
            "time_bucket",
        ],
        ascending=True,
    )
    monthly_trips_weather = monthly_trips_weather.rename(
        columns={
            "hvfhs_license_num": "trips",
            "trip_miles": "avg_distance",
        },
    )

    monthly_trips_weather.to_parquet("s3://dask-results/result.pq", compute=True)

    end = time.time()

    return end - start

In [37]:
for _ in range(1):
    future = client.submit(get_monthly_travels_weather)
    total_time = future.result()
    client.restart()
    print("Total time for IO and compute:", total_time)

Total time for IO and compute: 900.1462795734406


In [38]:
client.close()
cluster.close()

Terminated dask-64a5368e-worker-f84e5b15 (i-00c6467b960e4d53b)
Terminated dask-64a5368e-worker-226adae8 (i-0d8c2a80a2824f229)
Terminated dask-64a5368e-worker-ec933d21 (i-0e3ea90c1689e930d)
Terminated dask-64a5368e-worker-5f8cb338 (i-0027a7a478ec1e6de)
Terminated dask-64a5368e-scheduler (i-045352febc6cc424a)
